In [ ]:
import tweepy
from tqdm.notebook import tqdm
from datetime import datetime
import webbrowser

# Dead following unfollower
Automatically opens browser and lets you unfollow dead twiiter accounts.
Get new twitter credentials from https://developer.twitter.com/en first
Set your user id, and expiry date

In [ ]:
id = "yakinavault"
follower_expiry_date = "2021-06-01 +0000"

# twitter api creds
consumer_key = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
consumer_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
access_token = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
access_token_secret = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"

In [ ]:


auth = tweepy.OAuthHandler(
        consumer_key, consumer_secret)
auth.set_access_token(access_token,
                          access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
users = []
users_lta = {}


In [ ]:
for user in tqdm(tweepy.Cursor(api.friends, screen_name=id, count=200).items()):
    if user.screen_name not in users:
        users.append(user.screen_name)
print("Total following:", len(users))

In [ ]:
def last_tweet_at(user_id):
    tweet = api.user_timeline(id = user_id, count = 1)
    if len(tweet) > 0:
        return tweet[0]._json["created_at"]
    else:
        return None

In [ ]:
for user in tqdm(users):
    if user not in users_lta:
        try:
            lta = last_tweet_at(user)
        except tweepy.TweepError:
            print("Error: ", user)

        if lta is not None:
            users_lta[user] = lta

print("Total users with last tweets:", len(users_lta))

In [ ]:
def parse_twitter_time(time_str):
    return datetime.strptime(time_str, '%a %b %d %H:%M:%S %z %Y')

In [ ]:
sorted_lta = sorted(users_lta.items(), key=lambda x: parse_twitter_time(x[1]) )

In [ ]:
to_be_unfollowed = list(filter(lambda x: parse_twitter_time(x[1]) < datetime.strptime(follower_expiry_date, "%Y-%m-%d %z"), sorted_lta))
print("Total users to be unfollowed:", len(to_be_unfollowed))

In [ ]:
for user, _ in to_be_unfollowed:
    webbrowser.open_new_tab("https://twitter.com/{}".format(user))